# Creating a Simple Agent with Tracing

In [1]:
import dotenv
import os

from openai import OpenAI

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

We are up and running!


In [2]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

Create a simple Nutrition Assistant Agent

In [4]:
nutrition_agent = Agent(
    name="NutritionAgent",
    instructions="""
    You are a helpful assistant giving out nutrition advice.
    You give concise answers.""")

Let's execute the Agent:

In [5]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

RunResult:
- Last agent: Agent(name="NutritionAgent", ...)
- Final output (str):
    Overall, yes—bananas are a healthy, convenient fruit to include in a balanced diet.
    
    Key nutrients per medium banana (about 105 kcal):
    - Potassium: supports heart and muscle function
    - Vitamin B6: important for metabolism and brain health
    - Vitamin C: antioxidant, immune support
    - Fiber: aids digestion; resistant starch when greener
    
    Benefits:
    - Quick energy from natural sugars
    - May aid digestion and fullness
    
    Things to consider:
    - Higher sugar and calories than some fruits; portion if you’re watching sugar intake
    - People with kidney disease or potassium-sensitive conditions should monitor intake with a clinician
    - Ripe vs. unripe: greener = more resistant starch; riper = sweeter, faster-digested sugars
    
    Bottom line: good daily option in moderation as part of a varied diet.
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail resu

Streaming the answer to the screen, token by token

In [6]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

Bananas are nutritious and generally healthy when eaten as part of a balanced diet.

Key points:
- Good nutrients: potassium, vitamin B6, vitamin C, fiber.
- Health benefits: supports heart health, digestion, and convenient energy for workouts.
- Varied by ripeness: unripe has more resistant starch; ripe has more sugars.
- Calories: about 90–110 kcal per medium fruit.
- Considerations: portion if you’re watching sugar intake or have kidney issues; people with diabetes should count carbs.

Overall: a healthy, convenient fruit to include regularly.

_Good Job!_